# PMF–ALS Experiment: Testing Subspace Invariance Theory

## Core Theoretical Claim

**When ALS is initialized in the subspace spanned by specific singular vectors of Y, it will:**
1. **Stay in that subspace** (subspace invariance)
2. **Converge to the stationary point** obtained by soft-thresholding the corresponding singular values

## Mathematical Setup

- **MAP Objective:** $L(U,V) = \frac{1}{2}\|Y - UV^\top\|_F^2 + \frac{\lambda}{2}(\|U\|_F^2 + \|V\|_F^2)$
- **Global Minimizer:** $U^* = F_R \text{diag}\big(\sqrt{(\sigma_i - \lambda)_+}\big)$, $V^* = G_R \text{diag}\big(\sqrt{(\sigma_i - \lambda)_+}\big)$
- **Soft-thresholding:** $(x)_+ = \max(x,0)$

## 1. Setup: Parameters and Data Generation

In [245]:
import numpy as np
import numpy.linalg as npl

# Fixed seed for reproducibility
np.random.seed(42)

# === EXPERIMENT PARAMETERS ===
# Matrix dimensions (smaller for clarity)
m, n = 10, 8
true_rank = 4

# Clean integer singular values (hardcoded for interpretability)
# add some noise to singular values
singular_values = [10, 8, 4, 1] 

# ALS parameters
R = 3                  # Latent factorization rank
lambda_reg = 2         # Regularization parameter

# Subspace selection
top_indices = [0, 1, 2]        # Top 3 singular vectors
alt_indices = [1, 2, 3]        # Alternative 3 (overlapping set)

print("=== EXPERIMENT SETUP ===")
print(f"Matrix size: {m} × {n}")
print(f"Singular values: {singular_values}")
print(f"Regularization λ = {lambda_reg}")
print(f"Latent rank R = {R}")
print(f"Top subspace indices: {top_indices}")
print(f"Alt subspace indices: {alt_indices}")

=== EXPERIMENT SETUP ===
Matrix size: 10 × 8
Singular values: [10, 8, 4, 1]
Regularization λ = 2
Latent rank R = 3
Top subspace indices: [0, 1, 2]
Alt subspace indices: [1, 2, 3]


In [246]:
# Generate synthetic matrix with known structure
# Y = U_true * S_true * V_true^T
U_true, _ = np.linalg.qr(np.random.randn(m, true_rank))
V_true, _ = np.linalg.qr(np.random.randn(n, true_rank))
Y = U_true @ np.diag(singular_values) @ V_true.T

# Compute SVD of Y (should recover our structure)
U_svd, s_svd, Vt_svd = npl.svd(Y, full_matrices=False)
V_svd = Vt_svd.T

print("\n=== DATA GENERATION VERIFICATION ===")
print(f"Y shape: {Y.shape}")
print(f"Recovered singular values: {s_svd[:4].round(6)}")
print(f"Expected singular values:  {singular_values}")
print(f"Match: {np.allclose(s_svd[:4], singular_values)}")


=== DATA GENERATION VERIFICATION ===
Y shape: (10, 8)
Recovered singular values: [10.  8.  4.  1.]
Expected singular values:  [10, 8, 4, 1]
Match: True


## 2. Theory: Expected Solutions via Soft-Thresholding

In [247]:
def soft_threshold(sigma, lam):
    """Apply soft-thresholding: max(sigma - lambda, 0)"""
    return np.maximum(sigma - lam, 0.0)

def compute_map_solution(U_svd_subset, V_svd_subset, sigma_subset, lam):
    """Compute MAP solution U*, V* for given subspace"""
    # Apply soft-thresholding to get target scalings
    target_scales = np.sqrt(soft_threshold(sigma_subset, lam))
    
    # Form MAP solution matrices
    U_target = U_svd_subset @ np.diag(target_scales)
    V_target = V_svd_subset @ np.diag(target_scales)
    
    return U_target, V_target, target_scales

# Extract subspace components
U_svd_top = U_svd[:, top_indices]
V_svd_top = V_svd[:, top_indices] 
sigma_top = s_svd[top_indices]

U_svd_alt = U_svd[:, alt_indices]
V_svd_alt = V_svd[:, alt_indices]
sigma_alt = s_svd[alt_indices]

# Compute theoretical MAP solutions
U_target_top, V_target_top, scales_top = compute_map_solution(U_svd_top, V_svd_top, sigma_top, lambda_reg)
U_target_alt, V_target_alt, scales_alt = compute_map_solution(U_svd_alt, V_svd_alt, sigma_alt, lambda_reg)

print("=== SOFT-THRESHOLDING ANALYSIS ===")
print(f"Regularization λ = {lambda_reg}")
print()
print(f"Top subspace (indices {top_indices}):")
print(f"  Singular values: {sigma_top.round(2)}")
print(f"  After thresholding: {soft_threshold(sigma_top, lambda_reg).round(2)}")
print(f"  Target scales: {scales_top.round(3)}")
print()
print(f"Alt subspace (indices {alt_indices}):")
print(f"  Singular values: {sigma_alt.round(2)}")
print(f"  After thresholding: {soft_threshold(sigma_alt, lambda_reg).round(2)}")
print(f"  Target scales: {scales_alt.round(3)}")

=== SOFT-THRESHOLDING ANALYSIS ===
Regularization λ = 2

Top subspace (indices [0, 1, 2]):
  Singular values: [10.  8.  4.]
  After thresholding: [8. 6. 2.]
  Target scales: [2.828 2.449 1.414]

Alt subspace (indices [1, 2, 3]):
  Singular values: [8. 4. 1.]
  After thresholding: [6. 2. 0.]
  Target scales: [2.449 1.414 0.   ]


## 3. Initialize: Set ALS Starting Points

In [248]:
def initialize_in_subspace(U_svd_subset, V_svd_subset, scale_range=(0.5, 2.0)):
    """Initialize U0, V0 in given subspace with random scalings"""
    R = U_svd_subset.shape[1]
    
    # Random scaling factors
    scales_U = np.random.uniform(scale_range[0], scale_range[1], R)
    scales_V = np.random.uniform(scale_range[0], scale_range[1], R)
    
    # Initialize in subspace with random scalings
    U0 = U_svd_subset @ np.diag(scales_U)
    V0 = V_svd_subset @ np.diag(scales_V)
    
    return U0, V0, scales_U, scales_V

# Initialize ALS starting points in each subspace
U0_top, V0_top, init_scales_U_top, init_scales_V_top = initialize_in_subspace(U_svd_top, V_svd_top)
U0_alt, V0_alt, init_scales_U_alt, init_scales_V_alt = initialize_in_subspace(U_svd_alt, V_svd_alt)

print("=== INITIALIZATION ===")
print(f"Top subspace - Initial U scales: {init_scales_U_top.round(2)}")
print(f"Top subspace - Initial V scales: {init_scales_V_top.round(2)}")
print(f"Alt subspace - Initial U scales: {init_scales_U_alt.round(2)}")
print(f"Alt subspace - Initial V scales: {init_scales_V_alt.round(2)}")
print()
print("Key insight: ALS should adjust these random scalings to the optimal target scales!")

=== INITIALIZATION ===
Top subspace - Initial U scales: [1.66 1.24 1.28]
Top subspace - Initial V scales: [1.14 0.54 0.66]
Alt subspace - Initial U scales: [0.55 1.45 0.97]
Alt subspace - Initial V scales: [1.26 1.86 0.87]

Key insight: ALS should adjust these random scalings to the optimal target scales!


## 4. Run: Execute ALS Algorithm

In [249]:
def als_algorithm(Y, U0, V0, lam, max_iterations=100, tolerance=1e-10, verbose=True):
    """Run ALS algorithm with convergence tracking"""
    U, V = U0.copy(), V0.copy()
    obj_prev = np.inf
    
    for iteration in range(max_iterations):
        # ALS updates
        VtV_reg = V.T @ V + lam * np.eye(V.shape[1])
        U = Y @ V @ npl.inv(VtV_reg)
        
        UtU_reg = U.T @ U + lam * np.eye(U.shape[1])
        V = Y.T @ U @ npl.inv(UtU_reg)
        
        # Compute objective
        residual = Y - U @ V.T
        obj = 0.5 * np.sum(residual**2) + 0.5 * lam * (np.sum(U**2) + np.sum(V**2))
        
        # Check convergence
        rel_change = abs(obj_prev - obj) / max(1.0, obj_prev)
        if rel_change < tolerance:
            if verbose:
                print(f"  Converged after {iteration + 1} iterations (rel_change: {rel_change:.2e})")
            break
        
        obj_prev = obj
    
    return U, V, obj, iteration + 1

print("=== RUNNING ALS ===")
print("Top subspace:")
U_result_top, V_result_top, obj_top, iters_top = als_algorithm(Y, U0_top, V0_top, lambda_reg)

print("\nAlt subspace:")
U_result_alt, V_result_alt, obj_alt, iters_alt = als_algorithm(Y, U0_alt, V0_alt, lambda_reg)

print(f"\nFinal objectives:")
print(f"  Top subspace: {obj_top:.6f}")
print(f"  Alt subspace: {obj_alt:.6f}")

=== RUNNING ALS ===
Top subspace:
  Converged after 12 iterations (rel_change: 3.35e-11)

Alt subspace:
  Converged after 9 iterations (rel_change: 3.29e-11)

Final objectives:
  Top subspace: 38.500000
  Alt subspace: 70.500000


/var/folders/d2/m1r5k9xs77n9kj281b15qkgw0000gn/T/ipykernel_11689/3752252611.py:19: RuntimeWarning: invalid value encountered in scalar divide
  rel_change = abs(obj_prev - obj) / max(1.0, obj_prev)


## 5. Validate: Check Convergence Results

In [250]:
def validate_convergence(U_result, V_result, U_target, V_target, test_name, 
                        tol_subspace=1e-4, tol_reconstruction=1e-4):
    """Comprehensive validation of ALS convergence"""
    
    # 1. Reconstruction error (should be ~0 if found correct solution)
    reconstruction_result = U_result @ V_result.T
    reconstruction_target = U_target @ V_target.T
    reconstruction_error = np.linalg.norm(reconstruction_result - reconstruction_target, 'fro')
    
    # 2. Scaling consistency via Gram matrices
    gram_U_error = np.linalg.norm(U_result.T @ U_result - U_target.T @ U_target, 'fro')
    gram_V_error = np.linalg.norm(V_result.T @ V_result - V_target.T @ V_target, 'fro')
    
    # Overall pass/fail
    reconstruction_ok = reconstruction_error <= tol_reconstruction
    overall_pass = reconstruction_ok
    
    status = "PASS ✓" if overall_pass else "FAIL ✗"
    
    print(f"\n{test_name}")
    print(f"  Status: {status}")
    print(f"  Reconstruction error: {reconstruction_error:.2e}")
    print(f"  Gram matrix errors: U={gram_U_error:.2e}, V={gram_V_error:.2e}")
    
    return {
        'status': status,
        'overall_pass': overall_pass,
        'reconstruction_error': reconstruction_error,
        'gram_U_error': gram_U_error,
        'gram_V_error': gram_V_error
    }

print("=== VALIDATION RESULTS ===")

# Test 1: Top subspace should converge to global minimum
result_top = validate_convergence(U_result_top, V_result_top, U_target_top, V_target_top, 
                                 "Test 1: Top subspace → Global minimum")

# Test 2: Alt subspace should converge to saddle point
result_alt = validate_convergence(U_result_alt, V_result_alt, U_target_alt, V_target_alt,
                                 "Test 2: Alt subspace → Saddle point")

# Diagnostic: Check if alt escaped to top solution (should be FAIL)
result_escape = validate_convergence(U_result_alt, V_result_alt, U_target_top, V_target_top,
                                   "Diagnostic: Alt → Top (should FAIL)")

=== VALIDATION RESULTS ===

Test 1: Top subspace → Global minimum
  Status: PASS ✓
  Reconstruction error: 8.76e-06
  Gram matrix errors: U=4.38e-05, V=2.63e-05

Test 2: Alt subspace → Saddle point
  Status: PASS ✓
  Reconstruction error: 6.62e-06
  Gram matrix errors: U=2.65e-05, V=1.32e-05

Diagnostic: Alt → Top (should FAIL)
  Status: FAIL ✗
  Reconstruction error: 8.00e+00
  Gram matrix errors: U=4.90e+00, V=4.90e+00


## 6. Summary: Final Results

In [251]:
print("\n" + "="*50)
print("EXPERIMENT SUMMARY")
print("="*50)

print(f"\nMatrix: {m}×{n}, Rank: {true_rank}, Regularization: λ={lambda_reg}")
print(f"Singular values: {singular_values}")
print(f"Latent rank: R={R}")

print(f"\nSubspace Selection:")
print(f"  Top indices {top_indices}: σ = {sigma_top.round(1)} → target scales = {scales_top.round(2)}")
print(f"  Alt indices {alt_indices}: σ = {sigma_alt.round(1)} → target scales = {scales_alt.round(2)}")

print(f"\nTheoretical Prediction:")
print(f"  ALS should stay within initialized subspace")
print(f"  ALS should find MAP scaling via soft-thresholding formula")

print(f"\nExperimental Results:")
print(f"  Test 1 (Top → Global): {result_top['status']}")
print(f"  Test 2 (Alt → Saddle): {result_alt['status']}")
print(f"  Diagnostic (Alt → Top): {result_escape['status']} (expected to fail)")

both_pass = result_top['overall_pass'] and result_alt['overall_pass']
print("="*50)


EXPERIMENT SUMMARY

Matrix: 10×8, Rank: 4, Regularization: λ=2
Singular values: [10, 8, 4, 1]
Latent rank: R=3

Subspace Selection:
  Top indices [0, 1, 2]: σ = [10.  8.  4.] → target scales = [2.83 2.45 1.41]
  Alt indices [1, 2, 3]: σ = [8. 4. 1.] → target scales = [2.45 1.41 0.  ]

Theoretical Prediction:
  ALS should stay within initialized subspace
  ALS should find MAP scaling via soft-thresholding formula

Experimental Results:
  Test 1 (Top → Global): PASS ✓
  Test 2 (Alt → Saddle): PASS ✓
  Diagnostic (Alt → Top): FAIL ✗ (expected to fail)
